In [ ]:
import numpy as np
import pennylane as qml
import argparse
qml.QubitStateVector = qml.StatePrep

from variational_lse_solver import VarLSESolver


"""
To run this script you need the following packages:
- PennyLane (but the latest version does not have the QubitStateVector class (which is needed), so we redefine it above)
- Numpy < 2: need to install something like numpy 1.26.4
- Python 3.12
- torch 2.2.2
"""


''

In [12]:

# unitary representation of Paulis
I_ = np.array([[1.0, 0.0], [0.0, 1.0]])
X_ = np.array([[0.0, 1.0], [1.0, 0.0]])
Y_ = np.array([[0.0, -1.j], [1.j, 0.0]])
Z_ = np.array([[1.0, 0.0], [0.0, -1.0]])

a = ["III", "XZI", "XII"]
b = np.ones(8)/np.sqrt(8)



In [13]:
lse = VarLSESolver(a, 
                   b, 
                   coeffs=[1.0, 0.2, 0.2], 
                   method="hadamard", 
                   local=False, 
                   lr=0.01, steps=50) 


solution, _ = lse.solve()

Epoch 1/1: 100%|██████████| 50/50 [00:01<00:00, 31.84it/s, best loss=0.592, last improvement in step=49, loss=0.592]


In [14]:
# compute normalized classical solution for comparison, therefore first re-compose system matrix A
A = 1.0 * np.kron(I_, np.kron(I_, I_)) + 0.2 * np.kron(X_, np.kron(Z_, I_)) + 0.2 * np.kron(X_, np.kron(I_, I_))
classical_solution = np.linalg.solve(A, b)
normalized_classical_solution = np.square(classical_solution / np.linalg.norm(classical_solution))

# print and compare solutions
print('\nQuantum:\n|', end='')
for s in solution:
    print(f' {s:.4f} |', end='')
print('\nClassical:\n|', end='')
for s in normalized_classical_solution:
    print(f' {s:.4f} |', end='')
print()


Quantum:
| 0.4490 | 0.3324 | 0.1218 | 0.0901 | 0.0030 | 0.0023 | 0.0008 | 0.0006 |
Classical:
| 0.0845 | 0.0845 | 0.1655 | 0.1655 | 0.0845 | 0.0845 | 0.1655 | 0.1655 |
